In [2]:
import os
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
#options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path ='C:/Users/kimjunseok/Crawling/chromedriver.exe'
driver = webdriver.Chrome(ChromeDriverManager().install())  # chromedriver 열기


def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    place_infos = ['강남 맛집']

    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)

    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page'))
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()
    #파이썬에서는 자동으로 글로벌 변수 __name__ 을 __main__ 이라고 할당한다. 
    #따라서, 해당 프로그램에서는 if 문을 사용해서 __name__ 이 __main__ 인지 확인하여
    #맞으면 해당 if 문에 있는 내용을 돌리도록 하면 그것이 메인 프로그램에서 실제로 돌아가는 코드가 되겠다. 
    # if __name__=="__main__"이라는 조건문을 넣어주고 그 아래는 직접 실행시켰을 때만 실행되길 원하는 코드들을 넣어주는 것



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\kimjunseok\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache


##### 0
#### 광고쓲
또 가고시픈곳 넘 분위기 맛 조아유!/5
원테이블로 해서 프라이빗하게 다른 손님 없이 밥 먹을 수 있어서 좋았어요. 코스로 음식도 잘 나오고 분위기도 좋아서 다음 기념일이나 좋은 일 있을 때 또 방문하고 싶습니다./5
no review in crawling
no review in crawling
#### 광고삼시세끼 셰프반찬
반찬 다 맛있어서 자주 주문하고 있어요!/5
추천~하고 가요ㅎ^^/4
no review in crawling
no review in crawling
#### 호족반 청담
들기름 메밀국수 꼬소하고 트러플감자전 맛있긴 한대 느끼하긴 함..수육은 보통/3
웨이팅만 안했다면.. 쏘쏘/3
오픈시간 보다 일찍 갔지만 120분 기다리고 먹었어요 몽탄 금돼지 등등 웬만한 웨이팅 맛집들 다 기다릴 가치가 있었는데 여기는 제겐 첫방문=마지막 방문이 되었습니다..  들기름 국수 - 아무맛이 안나요 마늘밥 - 버터맛만 나요 육회 - 작아요.. 아보카도도 적어요 감자전 - 일행은 맛있다 했으나 저는 모르겠어요 갈비 - 짜요.. 많이 짜요  확진 휴무 이후.../1
가장 중요한 웨이팅은 공휴일 기준 1시 49분에 예약 걸어서 2시간 13분 걸렸습니다. 저희는 일보고 온지라 그렇게 불편함은 못 느꼈어요. NY 양념 갈비, 들기름 메밀국수, 마늘 항정 수육 먹었고 모두 평균에서 평균 이하의 맛이었어요. 유명하다고 해서 갔는데 'MZ세대 감성 참 어렵다' 라는 느낌 많이 받고 왔네요 ㅎㅎ   친절도는 5점입니다. 다들 매우.../2
맛도 있고 테이블도 다 좋은데  앱으로 웨이팅 3시간 넘게 했는데 3분만에 안온다고 취소 해서 10분 더 기다림 앱 예약시스템이야 알고 가면 기다리는 건 힘들지 않을듯 처음엔 음 맛있다!는 생각은 했지만 갔다와서 든 생각은 이 가격에 맛도 양도 그냥 그래서 다시 먹으러 갈지가 의문..! 하지만 한 번 가보는거 추천함./3
웨이팅하지 않고 먹어서 그런가 난 5점../5
비싼거 빼고 다 좋고 맛있음/4
대기 안 하

다운타우너 주문하고 얼른 포장해오면 꿀!/5
맛은 있는데 줄이 너무 길어서…가성비야 이런곳에서 찾으면 안되는거고 ㅋㅋ 암튼 한번은 인스타에 올리는 용도(?)로 가는데 두번은 못가겠음 ㅋㅋㅋㅋ/4
처음 빵 한입 먹었을때 시장 도넛인가 했는데 크림이랑 먹으니 대대대대 존맛 ㅠㅠ! 크림 너무 맛있어요/5
망고맛 크림이 들어있는 케이크! 안에 망고 슬라이스나 망고 과육은 없음! 하지만 크림이 느끼하지 않고 부드러워서 좋음!/4
도넛이 인기있지만 케이크 정말 맛있어요! /4


KeyboardInterrupt: 

In [3]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
#options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path ='C:/Users/kimjunseok/Crawling/chromedriver.exe'
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기


def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    place_infos = ['강남 맛집']

    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)

    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """
    try:

        while_flag = False
        for i, place in enumerate(place_lists):
            # 광고에 따라서 index 조정해야함
            #if i >= 3:
             #   i += 1

            place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
            place_address = place.select('.info_item > .addr > p')[0].text  # place address

            detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
            driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
            driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
            sleep(1)

            print('####', place_name)

            # 첫 페이지
            extract_review(place_name)

            # 2-5 페이지
            idx = 3

            try:
                page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
                for i in range(page_num-1):
                    # css selector를 이용해 페이지 버튼 누르기
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")

            # 그 이후 페이지
            while True:
                idx = 4
                try:
                    page_num = len(driver.find_elements_by_class_name('link_page'))
                    for i in range(page_num-1):
                        driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                        sleep(1)
                        extract_review(place_name)
                        idx += 1
                    driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                    sleep(1)
                    extract_review(place_name) # 리뷰 추출
                except (NoSuchElementException, ElementNotInteractableException):
                    print("no review in crawling")
                    break

            driver.close()
            driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환
    except NoSuchElementException:
        print('not found')


def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()
    #파이썬에서는 자동으로 글로벌 변수 __name__ 을 __main__ 이라고 할당한다. 
    #따라서, 해당 프로그램에서는 if 문을 사용해서 __name__ 이 __main__ 인지 확인하여
    #맞으면 해당 if 문에 있는 내용을 돌리도록 하면 그것이 메인 프로그램에서 실제로 돌아가는 코드가 되겠다. 
    # if __name__=="__main__"이라는 조건문을 넣어주고 그 아래는 직접 실행시켰을 때만 실행되길 원하는 코드들을 넣어주는 것

##### 0
#### 광고진도집
여기 사람들 너무 많아요ㅠㅠ 너무 맛있어서 그런거겠죠?? 여름에는 더워서 힘들고 겨울에는 추워서 ㅠㅠㅠㅠ 담에는 일찍 방문 해야 겠어요!!!/5
점심이든 저녁이든 항상 사람 많은 이곳, 정말 맛있답니다!/5
요즘 같이 쌀쌀한 날 해물탕 먹고 싶어서 찾다가 리뷰도 좋아서 가서 먹었어요. 너무 얼큰하고 맛있어요ㅠㅠ담에 부모님이랑 또 갈게요!/5
/5
점심에 바글바글. 맛있어요/5
회식하러 자주 갑니다./5
식사메뉴 저렴하고 맛나요/5
점심 먹으러 갔는데 해물탕도 적당히 얼큰하고 무엇보다도 싱싱한 것 같아 너무 좋았어요. 친절하신 사장님 덕분에 자주 갈 것 같아요~ㅎㅎ/5
해물로스 최애메뉴. 여긴 항상 사람이 많은 것 같아요./5
점심 먹으러 몇번 갔었는데 최곱니다/5
저녁의 해물로스, 점심의 철판밥/5
물회도 낙지찌개도 너무 맛있고 사장님도 너무 친절하셔요 재방문 의사 100입니다 /5
점심메뉴 굿~/4
/3
/3
/4
/3
맛은 있음 그런데 가격대비  양이 정말 작음. 아무리 가계세가 비싼 동네라지만 가격이 정말 너무 비쌈. 청담동수준의 가격임 맛만 보고 갈만하나 가격생각하면  안가고 싶음/3
저는 고임금 시대의 필수 아이템인 무인주문.결제 시스템, K키오스크 입니다.매장의 빠른 회전율로 매출이 증대되고  인건비 절감을 할수있는 국내 키오스크 시장 점유율 1위 업체  제품을 소개 드리고자 합니다. 연락 주시면 자세히 설명 드리겠습니다.H.P: 010-3569-1343/5
진짜 여기 정말정말 너무 맛있어요!!/5
지난주 직원들하고 낙지전골 먹었습니다. 이열치열!! 잘 먹었습니다./5
해물로스로 회식했던 곳!! 요즘 살~짝 느끼한게 땡기는게 회식한지 몇 주 지났는데도 또 생각나게 하네요~ 이번엔 친구들 꼬셔서 갈께요!!!!/5
쭈구미 상태도 싱싱하고 배불리 먹고 나왔습니다. 맛없으면 먹은것 같지도 않은데 맛있으면서도 또 들르고 싶은 곳이기도 하네요. 직원들과 함께 회식장소 추천합니다~/5
회사 근처에 잇어서 회식할때마다 가는데 리뷰

줄서서 먹을 만한 평냉집이네요.  만두 꼭 드셔보세요~~ 집에 갖고와서 식은 후 먹어도 고기냄새 없이 맛나네요/5
그저그런 정도/3
냉면자체만 본다면 수준급의 냉면. 단, 양이 넘 적었고, 냉면김치가 좀..;;;/4
만두가 최고존엄 대존맛이에여/4
맛 짱짱/5
/1
너무 맛난 곳 최고최고/5
/3
나오자마자 정신없이먹느라 냉면사진을 깜빡할 정도로 맛나요맛나♡ 만두꼭시켜드십시오들~~~♡/5
다른 평양냉면집보다 좀 짠편. 그래서 입맛에 맞다ㅋㅋㅋ수육 반접시 파는것도 좋음. /4
육수가 담백하고 깔끔해요. 편육과 만두는 쏘쏘/4
냉면이랑 만둣국이 정말 맛있어요!!!!! 냉면 처음 먹어봤을 땐 뭐지??하긴 했는데 먹다보니까 너무 맛있더라고요ㅎㅎ 완전 강추해요/5
국물 담백하고 특히 면이 좋았어요 향도 찐하고/4
주차 안내하시는 분 정말 불친절해요. 두번 다시 안 가고 싶네요. 맛도 다른 평양냉면에 비해 떨어집니다./1
뭔가 애매한 맛...그리고 은근 좁음/3
숟가락 5개 드립니다 /5
/3
강제 발렛파킹당하고 4000원 뜯김/1
평양냉면 좋아하시는 분이라면 꼭 가세요 대신 가격대가 좀 있음. 별4점인 이유는 발렛파킹이 불친절함~~/4
더할나위없이 깔끔한 맛 아주조금 심심한 느낌은 있다/4
짱맛 /5
와 찐이다 찐/5
평냉  어복쟁반  소주  최고다/5
메밀도 수입산, 육수 고기도 수입산인데 가격 무엇????? /1
가격이 정말 너무하네요. 아무리 논현동이라지만/1
평양냉면 중 제일 맛있고 수육은 입에서 녹아요/5
/4
평냉 맛있어요. 편육,만두도 괜찮았음./4
처음 먹어봤는데 나쁘지 않네요/4
여기 맹물이라고 하는 분들은 평소에 한번도 평양냉면 안먹다가 미슐랭 붙은거만 보고 온듯.. 육수만 먹으면 간 엄청 센편이고 면 풀어지면 밸런스 너무 좋습니다. 서비스 상관없고 웨이팅 싫으신 분들 오시면 딱이에요/5
냉면 육수 맛있어요 발레파킹 3000원.../4
언제부터인가 어복쟁반은 진미에서만 먹어요. 정말 맑은데 진한 어이없는 국물맛이랑 그 고기+소스의 기막힌 조화 때문에 

홍콩을 추억하는데애 의의있음. 딤섬이지만 쉽게 먹을 수 없는점은 아쉬움 ㅠㅠ/3
홍콩에서 먹던 그 맛!에는 약간 부족하지만 맛있어서 재방문했어요./5
/4
챠슈바오 번 맛 있어요. 나머진 soso/4
연잎밥 먹으러 자주 가요!/4
아 맛이 없는건 아닌데 뭔가 애매함/3
30분정도 대기했구요 너무 좋았어요 사람이 많아서 정신없긴했구요 물이 부족해서 여러번 요청했어요 사천식완탕은 사랑입니다    에그타르트도 너무 맛있었어요   딸이 무척 잘 먹었어요 /5
맛좋구친절해요/4
맛있음/4
딤섬 먹기위해 올만한 집! 점심 피크타임만 지나면 사람도 많이 없어서 대기 안하고 앉을 수 있다. 추천은 샤오마이 와 라이스롤. 다만 튀김류가 맛이 없다... 닭고기 춘권과 파이구 면은 정말 기대 이하 돈 아까움. 딱 딤섬만 먹으러 오세요./4
좋아요/5
/4
사람많고 자리도넓고 시끄럽지만 음식이 너무 맛있음  사진 못찍은게 아쉬움.. 완탕 존맛 딤섬종류별로 다맛있음 ㅠㅠ 재방문 1000퍼 맛집/4
/5
홍콩에서처럼 팀호완은 하가우가 제일 맛있네요. 완탕면은 홍콩 다른 식당에서 먹던 맛에 한참 못미쳐요... 완탕 피가 두꺼워서 밀가루맛이 났고 면도 특유의 꼬들꼬들함이 안느껴져서 많이 아쉬웠습니다. 국물도 별 맛이 안느껴지고요. 그래도 그리운 홍콩을 추억하며 먹을 수 있다는 점이 메리트네요./3
많이 기다리긴했지만 너무 맛났어요/5
사람 진짜 많고 시끄럽고 시장바닥같긴한데 직원분들은 다들 엄청 친절하시던데요...???? 리뷰들보고 불친절할까봐 걱정했는데 엄청엄청 친절하셔서 좋았고 음식도 개인적으로는 대만 팀호완보다 맛있었어요 차슈바오번이랑 샤오마이 하가우 완탕면 다 맛있었어요:)/5
서비스 진짜 별로고 너무 시끄럽고 정신이 하나도 없어서 시장바닥에서 먹는줄알았음/1
서비스 정말 안좋아요. 매장이 개판인데 배달은 왜 하시는 건가요? 프론트에 있는 직원은 손님 있는데서 짜증내고 손님들은 순서 안불렀다고 화내고 홀은 빨리 안치우고 (혹은 인원 부족이거나) 주방은 포장 손님 번호 부르는데 영수

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=91.0.4472.164)
